In [ ]:
// #r ".\binaries\bossspad.dll"
// #r ".\binaries\XNSEC.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
static var myDb = OpenOrCreateDatabase(@"C:\Databases\BoSSS_DB_Couette");
//static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\HC_ConvProb\BoSSS_DB" );
myDb.Sessions

In [ ]:
BoSSSshell.WorkflowMgm.Init("CouetteTemperatureDifference");

In [ ]:
BoSSSshell.WorkflowMgm.Sessions

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int Res) { 
        var xNodes =  GenericBlas.SinLinSpacing(0.0, 1.0, 0, Res + 1);
        var yNodes =  GenericBlas.SinLinSpacing(0.0, 1.0, 0,Res + 1);
        var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes,  type: CellType.Square_Linear);
        grd.EdgeTagNames.Add(1, "velocity_inlet_bottom");
        grd.EdgeTagNames.Add(2, "velocity_inlet_left");
        grd.EdgeTagNames.Add(3, "velocity_inlet_right");
        grd.EdgeTagNames.Add(4, "velocity_inlet_top"); // moving wall
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];            
         
           if(Math.Abs(x - 0) < 1e-8)  //left Inlet
              return 2;      
            if(Math.Abs(y - 0) < 1e-8) //bottom Wall
                return 1;     
            if(Math.Abs(x - 1) < 1e-8)  // right outlet
                return 3;      
            if(Math.Abs(y - 1) < 1e-8) //top Wall
                return 4;
            else throw new ArgumentOutOfRangeException();
        });
        myDb.SaveGrid(ref grd);
        return grd;
     }
 
 }

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double Th, double Tc, double Froude, double c_1) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");
           stw.WriteLine("  static public double VelX(double[] X) {");
           stw.WriteLine("    return X[1];");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double VelY(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double One(double[] X) {");
           stw.WriteLine("    return  1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double Zero(double[] X) {");
           stw.WriteLine("    return  0.0;");
           stw.WriteLine("  }");

        //    stw.WriteLine(" static public double InitialPressure(double[] X) { ");
        //    stw.WriteLine("         return  (-1.0) *  / ("+Froude+" * "+Froude+" * ("+Th+" - "+Tc+")+) * Log(("+Th+" - "+Tc+") * X[1] + "+Tc+") + "+c_1+";");
        //    stw.WriteLine("    }");
        //    stw.WriteLine("}"); 

           stw.WriteLine(" static public double InitialTemperature(double[] X) { ");
            stw.WriteLine("      ("+Th+" - "+Tc+") * X[1] + "+Tc+" ;");
            stw.WriteLine("    }");
             stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
   
    static public Formula Get_VelX(double Th, double Tc , double Froude,double c_1) {
        return new Formula("BoundaryValues.VelX", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }

    static public Formula Get_VelY(double Th, double Tc,double Froude,double c_1) {
        return new Formula("BoundaryValues.VelY", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }
    static public Formula Get_TemperatureHot(double Th, double Tc, double Froude,double c_1) {
        return new Formula("BoundaryValues.TemperatureHot",AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }

     static public Formula Get_TemperatureCold(double Th, double Tc,double Froude,double c_1) {
        return new Formula("BoundaryValues.TemperatureCold", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }

    static public Formula Get_One(double Th, double Tc, double Froude,double c_1) {
        return new Formula("BoundaryValues.One", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }
     static public Formula Get_Zero(double Th, double Tc, double Froude,double c_1) {
         return new Formula("BoundaryValues.Zero",AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
     }
    // static public Formula Get_InitialPressure(double Th, double Tc, double Froude,double c_1) {
    //     return new Formula("BoundaryValues.InitialPressure",AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    // }
    static public Formula Get_InitialTemperature(double Th, double Tc, double Froude,double c_1) {
        return new Formula("BoundaryValues.InitialTemperature",AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude,c_1));
    }
}

## Send and run jobs

In [ ]:
var controls = new List<XNSEC_Control>();

In [ ]:
int[] Resolutions = new int[]{ 2,3,4,5,6 }; 
int[] DGdegrees = new int[]{ 1,2 };
// int[] Resolutions = new int[]{ 3 }; 
// int[] DGdegrees = new int[]{ 2 };

var controls = new List<XNSEC_Control>();
double Th = 1.6; double Tc = 0.4;
double Ra = 1e1;
foreach(int dg in DGdegrees){
foreach(int Res in Resolutions) {

    var C = new  XNSEC_Control();
    C.SetDGdegree(dg);
    var nCells = (int) Math.Pow(2,Res+1);
    C.SetGrid(GridFactory.GenerateGrid(nCells));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Dgdegree", dg));
    // Solver configuration
    // ========================
    C.ProjectName = "CouetteFlowTempGrad_DG" + dg + "K" + Res;
    C.ProjectDescription = "Steady Low Mach couette flow with temperature gradient";
    C.AnalyticsolutionSwitch = true;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();
    C.PhysicalParameters.IncludeConvection = false;
    C.MatParamsMode = MaterialParamsMode.Constant;
    C.ChemicalReactionActive = false;
    C.EnableMassFractions = false;
    C.PhysicalParameters.IncludeConvection = true;
    C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;    
    C.SessionName = "NaturalConvection_k" + Res + "_DG" + dg; 

    // Parameters
    // ==============
    C.Reynolds = 10.0;
    C.Prandtl = 0.71;
    C.Froude = Math.Sqrt(2 * C.Prandtl * (1.6 - 0.4) / (1.6 + 0.4));
    C.PressureReferencePoint = new double[] { 0.5*1, 0.5 *1};

    C.Schmidt = 1.0;
    C.PenaltyViscMomentum = 1.0;
    C.PenaltyHeatConduction = 1.0;

    C.GravityDirection = new double[] { 0.0, 1.0, 0.0 };

    double Th = 1.6; // Adimensional temperature of the hot wall (moving wall)
    double Tc = 0.4; // Adimensional temperature of the cold wall

    double p0 = 1.0; // The thermodynamic pressure, constant for open systems
    double c_1 = p0 / (C.Froude * C.Froude * (Th - Tc)) * Math.Log((Th - Tc) * C.PressureReferencePoint[1] + Tc) * 1; // A constant dependent on the reference point for the pressure

  
    // boundary conditions
    // ===================

    C.AddBoundaryValue("velocity_inlet_top", VariableNames.VelocityX, BoundaryValueFactory.Get_VelX(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_top", VariableNames.VelocityY, BoundaryValueFactory.Get_VelY(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_top", VariableNames.Temperature, BoundaryValueFactory.Get_InitialTemperature(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_top", VariableNames.MassFraction0,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1));

    C.AddBoundaryValue("velocity_inlet_left", VariableNames.VelocityX, BoundaryValueFactory.Get_VelX(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_left", VariableNames.VelocityY, BoundaryValueFactory.Get_VelY(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_left", VariableNames.Temperature, BoundaryValueFactory.Get_InitialTemperature(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_left", VariableNames.MassFraction0,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1));

    C.AddBoundaryValue("velocity_inlet_right", VariableNames.VelocityX, BoundaryValueFactory.Get_VelX(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_right", VariableNames.VelocityY, BoundaryValueFactory.Get_VelY(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_right", VariableNames.Temperature, BoundaryValueFactory.Get_InitialTemperature(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_right", VariableNames.MassFraction0,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1)); 

    C.AddBoundaryValue("velocity_inlet_bottom", VariableNames.VelocityX, BoundaryValueFactory.Get_VelX(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_bottom", VariableNames.VelocityY, BoundaryValueFactory.Get_VelY(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_bottom", VariableNames.Temperature, BoundaryValueFactory.Get_InitialTemperature(Th, Tc,C.Froude,c_1));
    C.AddBoundaryValue("velocity_inlet_bottom", VariableNames.MassFraction0,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1)); 

 

     C.AddInitialValue(VariableNames.VelocityX,  BoundaryValueFactory.Get_Zero(Th, Tc,C.Froude,c_1)); 
     C.AddInitialValue(VariableNames.VelocityY,BoundaryValueFactory.Get_Zero(Th, Tc,C.Froude,c_1)); 
    // C.AddInitialValue(VariableNames.Pressure,BoundaryValueFactory.Get_InitialPressure(Th, Tc,C.Froude,c_1)); 
    C.AddInitialValue(VariableNames.Temperature,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1)); 
    C.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1)); 

    C.AddInitialValue(VariableNames.ThermodynamicPressure,BoundaryValueFactory.Get_One(Th, Tc,C.Froude,c_1)); 


    controls.Add(C);
}
}

Grid Edge Tags changed.


Error: Microsoft.CodeAnalysis.Scripting.CompilationErrorException: (28,7): error CS0201: Only assignment, call, increment, decrement, await, and new object expressions can be used as a statement
   at Microsoft.CodeAnalysis.Scripting.ScriptBuilder.ThrowIfAnyCompilationErrors(DiagnosticBag diagnostics, DiagnosticFormatter formatter)
   at Microsoft.CodeAnalysis.Scripting.ScriptBuilder.CreateExecutor[T](ScriptCompiler compiler, Compilation compilation, Boolean emitDebugInformation, CancellationToken cancellationToken)
   at Microsoft.CodeAnalysis.Scripting.Script`1.GetExecutor(CancellationToken cancellationToken)
   at Microsoft.CodeAnalysis.Scripting.Script`1.RunAsync(Object globals, Func`2 catchException, CancellationToken cancellationToken)
   at Microsoft.CodeAnalysis.Scripting.Script`1.RunAsync(Object globals, CancellationToken cancellationToken)
   at Microsoft.CodeAnalysis.CSharp.Scripting.CSharpScript.RunAsync[T](String code, ScriptOptions options, Object globals, Type glob

## Run Simulations

In [ ]:
BoSSSshell.ExecutionQueues.ForEach(q => Console.WriteLine(q));

MiniBatchProcessor client @C:\deploy-Mini


SlurmClient: jg11bano@lcluster2.hrz.tu-darmstadt.de, Slurm account: project01299


MS HPC client  HPCCLUSTER @DC2, @\\hpccluster\hpccluster-scratch\gutierrez\binaries


MS HPC client  HPCCLUSTER2 @DC2, @\\hpccluster\hpccluster-scratch\gutierrez\binaries


MS HPC client @DC2, @\\hpccluster\hpccluster-scratch\gutierrez\binaries


In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[2];
//MiniBatchProcessor.Server.StartIfNotRunning(true);

In [ ]:
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

In [ ]:
foreach(var c in controls) {
    string jobName       = c.SessionName;
    var oneJob           = new Job(jobName, solver);
    oneJob.NumberOfMPIProcs = 2;    
    oneJob.NumberOfThreads = 1;
    oneJob.SetControlObject(c);   
    oneJob.Activate(myBatch); 

    //c.RunBatch(myBatch);    
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

All jobs finished.


## Create convergence plots

In [ ]:
var myDb1 = OpenOrCreateDatabase(@"C:\Databases\CounterFlowFlame_StrainSweep");

Opening existing database 'C:\Databases\CounterFlowFlame_StrainSweep'.


In [ ]:
myDb1.Sessions[1].Timesteps[5].Fields

#0: Phi
#1: PhiDG
#2: VelocityX
#3: VelocityY
#4: Pressure
#5: MixtureFraction
#6: VelocityX@Phi
#7: VelocityY@Phi


In [ ]:
var Jobs = BoSSSshell.WorkflowMgm.AllJobs;

In [ ]:
Jobs.ForEach(s => Console.WriteLine("{0}:{1}", Jobs.IndexOf(s),s));

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
BoSSSshell.WorkflowMgm.hConvergence.Update();


In [ ]:
Tab.GetColumnNames()

(empty)

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Temperature","Grid:hMin","Grid:NoOfCells" ,"L2Error_Temperature");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=36 );

Error: System.ArgumentException: Column "DGdegree:Temperature" does not exist.
   at BoSSS.Application.BoSSSpad.TableExtensions.ExtractColumns(DataTable Tab, String[] ColumnNames) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\TableExtensions.cs:line 674
   at Submission#25.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
//Tab2.Print();
var ErrorPlot = Tab2.ToPlot("Grid:hMin", "L2Error_Temperature","DGdegree:Temperature");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.TableExtensions.GetColumnNames(DataTable tab) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\TableExtensions.cs:line 524
   at BoSSS.Application.BoSSSpad.TableExtensions.ToPlot(DataTable Tab, String ColName_ForXValues, String ColName_ForYValues, String[] ColName_GroupSelection) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\TableExtensions.cs:line 972
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
ErrorPlot.PlotNow()

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.GnuplotExtensions.ToGnuplot(Plot2Ddata _2DData, GnuplotPageLayout layout) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\GnuplotExtensions.cs:line 274
   at BoSSS.Application.BoSSSpad.BoSSSpadGnuplotExtensions.PlotNow(Plot2Ddata _2DData) in C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\BoSSSpadGnuplotExtensions.cs:line 89
   at Submission#27.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:

//Tab.GetColumnNames().ForEach(f=>Console.WriteLine(f));